## 1. Why concurrency? (parallel vs. interleaved work)

Python programs often *feel* slow not because the CPU is overloaded but because they **wait**—for disk, network, or user input.  Concurrency lets a single program make useful progress while some tasks wait.

**CPU‑bound vs. I/O‑bound**
* *CPU‑bound* tasks (image processing, math) are limited by arithmetic speed.
* *I/O‑bound* tasks (web scraping, file uploads) spend most time blocked on the operating system.

Concurrency can mean:
* **Parallelism** – truly running *at the same time* on multiple cores or CPUs.
* **Interleaving** – rapidly switching tasks on a single core so each makes progress.

In CPython, the GIL restricts *parallel* Python bytecode execution by threads, but I/O interleaving (threads or `asyncio`) still yields large speed‑ups.

```python
import time, requests, threading
urls = ['https://example.com']*5

def fetch(url):
    requests.get(url)

start=time.perf_counter()
for u in urls: fetch(u)
print('sequential', time.perf_counter()-start)

start=time.perf_counter()
threads=[threading.Thread(target=fetch,args=(u,)) for u in urls]
for t in threads: t.start()
for t in threads: t.join()
print('concurrent', time.perf_counter()-start)
```

### Quick check

1. True / False Concurrency always speeds up CPU‑heavy number crunching in CPython.

2. Which task is likely **I/O‑bound**?
  a. multiplying two 1000×1000 matrices   b. downloading 100 web pages

<details><summary>Answer key</summary>

1. **False** – GIL prevents parallel bytecode; use `multiprocessing`.
2. **b**.

</details>

## 2. The GIL in plain English

The **Global Interpreter Lock** is a CPython mutex that allows only **one thread** to execute Python bytecode at any instant.  It simplifies memory management but limits parallelism.

Key points:
* Native C extensions like NumPy can **release the GIL** while crunching, enabling parallel CPU use.
* I/O operations release the GIL during system calls → threads are still useful for network/disk.
* Alternate interpreters: PyPy still has a GIL; Jython and IronPython don’t, but less ecosystem support.

```python
import threading, time
def cpu_bound():
    sum(i*i for i in range(10_000_00))

t0=time.perf_counter()
threads=[threading.Thread(target=cpu_bound) for _ in range(2)]
for t in threads:t.start()
for t in threads:t.join()
print('two threads:', time.perf_counter()-t0)

t0=time.perf_counter(); cpu_bound(); cpu_bound();
print('sequential :', time.perf_counter()-t0)
```

### Quick check

1. The GIL affects:
  a. threads   b. processes   c. both

2. True / False Network I/O releases the GIL so other threads can run.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 3. Threading fundamentals

*Creating*
`threading.Thread(target=func, args=(...,), daemon=False)` starts new OS thread.
*Joining*
`t.join()` blocks until thread finishes; always join to avoid zombie threads.
*Daemon threads*
Marked `daemon=True`, they **don’t block program exit**—good for background logging, bad if you need cleanup.

```python
import threading, time
def worker(n):
    print('start', n)
    time.sleep(1)
    print('done', n)

t = threading.Thread(target=worker, args=(1,))
t.start()
print('main continues')
t.join()
```

### Quick check

1. Daemon thread exits when main thread ends?
  a. yes   b. no

2. True / False `threading.active_count()` includes the main thread.

<details><summary>Answer key</summary>

1. **a**.
2. **True**.

</details>

## 4. Shared state & race conditions

Threads share memory → unsynchronised writes can interleave unpredictably.

Race condition demonstration: two threads incrementing a global counter often produce wrong total because `x += 1` is **read – modify – write** (three steps).

```python
import threading
counter = 0
def inc():
    global counter
    for _ in range(100_000):
        counter += 1
threads=[threading.Thread(target=inc) for _ in range(2)]
for t in threads:t.start()
for t in threads:t.join()
print('expected 200000 got', counter)
```

### Quick check

1. Race condition means result depends on:
  a. thread scheduling   b. deterministic order

2. True / False Using `+=` on list elements is atomic.

<details><summary>Answer key</summary>

1. **a**.
2. **False** – list ops may release GIL mid-way.

</details>